In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import overpass
import overpy
from shapely.ops import polygonize, polygonize_full, unary_union, linemerge
import shapely.geometry as geometry
import ee
import folium
import pprint
import json

In [3]:
ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AY0e-g7IQOv-x1rQZkbWzmPDsialABYdagZuX3k6Ujjwo7g-JN_zkuzPdRs



Successfully saved authorization token.


In [4]:
dataset = ee.ImageCollection("SKYSAT/GEN-A/PUBLIC/ORTHO/RGB")
land = ee.ImageCollection("AAFC/ACI")
info = dataset.first().getInfo()

In [5]:
a = dataset.toDictionary()
cols = dataset.first().propertyNames()
str = cols.join("\n")
#pprint.pprint(dataset.first().getInfo())

In [6]:
#print out the CRS information
print(info['bands'][0]['crs'])

PROJCS["UTM Zone 10, Northern Hemisphere", 
  GEOGCS["WGS 84", 
    DATUM["WGS_1984", 
      SPHEROID["WGS 84", 6378137.0, 298.257223563, AUTHORITY["EPSG","7030"]], 
      TOWGS84[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 
      AUTHORITY["EPSG","6326"]], 
    PRIMEM["Greenwich", 0.0, AUTHORITY["EPSG","8901"]], 
    UNIT["degree", 0.017453292519943295], 
    AXIS["Longitude", EAST], 
    AXIS["Latitude", NORTH], 
    AUTHORITY["EPSG","4326"]], 
  PROJECTION["Transverse_Mercator"], 
  PARAMETER["central_meridian", -123.0], 
  PARAMETER["latitude_of_origin", 0.0], 
  PARAMETER["scale_factor", 0.9996], 
  PARAMETER["false_easting", 500000.0], 
  PARAMETER["false_northing", 0.0], 
  UNIT["m", 1.0], 
  AXIS["x", EAST], 
  AXIS["y", NORTH]]


In [7]:
#get band names
bands = info['bands']
bandNames = []

#go through each band in the list and add it's name to the array
for x in bands:
    bandNames.append(x['id'])
    
#print out the array
bandNames

['R', 'G', 'B']

In [9]:
#get label names
labels = info['properties']
labelNames = []
#pprint.pprint(info)

#go through each key in the list and add it's name to the array
for x in labels.keys():
    labelNames.append(x)
    
#print out the array
labelNames

['resamplingMethod',
 'satelliteAzimuthAngleMin',
 'satelliteElevationAngleMax',
 'channelScaleOfLinearImages',
 'satelliteElevationAngleMin',
 'system:footprint',
 'productionID',
 'collectionEndTime',
 'satelliteAzimuthAngleMean',
 'productType',
 'satelliteName',
 'productionSystemVersion',
 'snaptoAlignmentConfidence',
 'system:time_end',
 'solarElevationAngle',
 'snaptoReferenceAssets',
 'system:time_start',
 'terrainBlendEpoch',
 'collectionType',
 'satelliteAzimuthAngleMax',
 'catalogID',
 'collectionStartTime',
 'satelliteElevationAngleMean',
 'system:asset_size',
 'solarAzimuthAngle',
 'system:index']

In [10]:
pprint.pprint(info['properties']['system:footprint'])

{'coordinates': [[-122.41488686359398, 38.74884444235594],
                 [-122.41489256725765, 38.74885242566341],
                 [-122.45397750490186, 38.75331064284916],
                 [-122.46190216975783, 38.71805043402878],
                 [-122.49757947299618, 38.722015751372616],
                 [-122.50559839848275, 38.68028630470562],
                 [-122.53814465503827, 38.56189533389399],
                 [-122.53813913073571, 38.56188699489921],
                 [-122.49864158347307, 38.55705144450406],
                 [-122.48781851654087, 38.59166815234509],
                 [-122.47776723046402, 38.590528816847055],
                 [-122.45949696637473, 38.58802473592073],
                 [-122.46857561977231, 38.55242950151375],
                 [-122.4685700917797, 38.552421224522256],
                 [-122.42900870408246, 38.54756188918841],
                 [-122.41651749385693, 38.60258788578131],
                 [-122.39768415642968, 38.666166423688